## Chapter 5   
### 순전파와 역전파   
문제를 계산그래프로 풀 때    
<li>왼쪽에서 오른쪽 : 순전파</li>   
<li>오른쪽에서 왼쪽 : 역전파</li>   
계산그래프의 특징으로는 <b>국소적 계산</b>이 있다.   
각각의 계산은 앞의 계산과는 상관없이 본인의 계산만 하면 된다. 따라서 역전파를 했을 때 미분을 효율적으로 계산할 수 있는 장점이 있다.   

### 연쇄법칙   
<i>합성 함수의 미분은 합성 함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있다.</i>   
$$ {\delta z \over \delta x} = {\delta z \over \delta t}{\delta t \over \delta x} $$   
이를 통해 계산 그래프의 역전파는 국소적 미분값들의 곱을 통해 최종적으로 미분값을 나타낼 수 있다.   

### 덧셈과 곱셈 노드의 역전파   
<b> 덧셈노드</b>
$ {\delta L \over \delta x }  $   
<br>
<b> 곱셈노드</b>
$ {\delta L \over \delta x }  {\delta x \over \delta y} $

덧셈노드의 경우 1을 곱하는 것이기 때문에 입력 신호 값이 필요 없지만, 곱셈노드의 경우 순방향 입력 신호 값이 필요하기 때문에 순전파의 입력 신호를 저장해둔다.   

In [1]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x*y

        return out
    
    def backward(self, dout): #역전파
        dx = dout * self.y
        dy = dout * self.x

        return dx, dy

apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer= MulLayer()

apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [2]:
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


In [4]:
class AddLayer:
    def __init__(self):
        pass
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout  * 1
        dy = dout * 1
        return dx, dy

In [5]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)

print(price)
print(dapple, dapple_num, dorange, dorange_num, dtax)


715.0000000000001
2.2 110.00000000000001 3.3000000000000003 165.0 650


계산 그래프에서 계층의 구현은 쉽게 구현할 수 있으며, 이를 사용해 복잡한 미분도 효율적으로 계산할 수 있다.   

ReLU 계층 구현하기   
$$
y = \begin{cases}x&(x>0)\\0&(x\le 0)\end{cases}\\
{\delta y \over \delta x} = \begin{cases}1&(x>0)\\0&(x\le 0)\end{cases}
$$   

Sigmoid 계층 구현하기   
$$
y = {1 \over 1 + exp(-x)}
$$   
이 때 위 식을 미분하는 것은 복잡하기 때문에 아래 그림처럼 나누어서 미분을 진행한다.     
![sigmoid_forward_graph](./images/sigmoid_foward_graph.png)   
### 역전파 순서   
1. '/'   
${\delta L \over \delta y} = -{1 \over x^2} = -y^2$   
2. '+' 그대로 통과   
$-{\delta L \over \delta y}{y^2}$   
3. 'exp' 지수함수의 미분은 $\log_eaa^x$
$-{\delta L \over \delta y}{y^2}exp(-x)$   
4. '$\times$' $y = -x \to {\delta y \over \delta x} = -1$   
${\delta L \over \delta y}{y^2}exp(-x)$   

In [ ]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx


$$
{\delta L \over \delta y}{y^2}exp(-x)\\
{1\over y}-1 = exp(-x)\\
={\delta L \over \delta y}{y^2}({1\over y}-1)\\
={\delta L \over \delta y}{y}(1-y)
$$

In [ ]:
import numpy as np

class Sigmoid:
    def __init__(self):
        self.out = None
    
    def forward(self, x):
        out = 1 / (1+np.exp(-x))
        self.out = out

        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx

In [ ]:
X = np.random.rnad(2)
W = np.random.rand(2, 3)
B = np.random.rand(3)

Y = np.dot(X, W) + B

>순전파 때 수행하는 행렬의 내적은 기하학에서 어파인 변환(Affine transformation)이라고 합니다.   위 책에서는 어파인 변환을 수행하는 처리를 'Affine 계층'이라는 이름으로 구현합니다.   

$
{\delta L \over \delta X} = {\delta L \over \delta Y}\cdot W^T\\
{\delta L \over \delta W} = X^T \cdot {\delta L \over \delta Y}
$   

X = (N, x) W = (x, w) B = (N, w) Y = (N, w)   
X에 대한 변화량을 구하려면 $Y\cdot W^T$      
(N, w) (w, x) = (N, x)   
W에 대한 변화량을 구하려면 $X^T \cdot Y$   
(x, N) (N, w) = (x, w)   

In [ ]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
    
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b

        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)

        return dx

### 왜 dx의 변화량을 구하는 것일까?   
x는 고정된 값이고 w가 수정되는 값이므로 dw를 구하는 것이 맞지 않나?   
근데 여기서 생각해봐야 하는 것이 여기에서 x는 데이터 자체 일수도 있지만 중간 계층의 결과물일 수도 있기 때문에 dx와 dw를 둘 다 구하는 것이 맞는 것 같다.   

###### Softmax layer   
$(a_1, a_2, a_3)$   normalize   $(y_1, y_2, y_3)$   
###### Cross Entropy Error   
$(y_1, y_2, y_3)$ & $(t_1, t_2, t_3)$ output $L$   

여기에서 softmax 계층의 역전파는 $(y_1-t_1, y_2-t_2, y_3-t_3)$를 나타낸다. 왜?   

![cross_entropy_error_backward](./images/cross_entropy_error_backward.png)

<b>계산 그래프 역전파의 규칙</b>   
1. 역전파의 초깃값은 1이다.   
2. $\times$ 노드의 역전파는 순전파 시의 입력값을 서로 바꿔 상류의 미분에 곱하고 하류로 흘린다.   
3. '+'노드에서는 상류에서 전해지는 미분을 그대로 흘린다.   
4. '/' 노드의 역전파는 상류에서 흘러온 값에 순전파 때의 출력을 제곱한 후 마이너스를 붙인 값을 곱해 하류로 전달한다.   
5. log 노드의 역전파는 $y = \log x \to {\delta y \over \delta x} = {1 \over x}$를 따른다.   
6. 순전파 때 갈라진 것은 역전파 때 값을 합치면 된다.   
7. 순전파 때 합쳐진 것은 역전파 때 값을 나누면 된다.   

위 규칙들을 이용하여 cross-entropy-error 에서 softmax 까지의 역전파가 $y_k - t_k$인 것을 알 수 있다.   

In [ ]:
import sys, os
sys.path.append(os.pardir)
from common.functions import *

class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
    
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size

        return dx

1. 미니배치   
    훈련 데이터 중 일부를 무작위로 가져오고 그 미니배치의 손실함수 값을 줄이는 것이 목표이다.   
2. 기울기 산출   
    미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구한다. 기울기는 손실 함수의 값을 작게 하는 방향을 제시한다.   
3. 매개변수 갱신   
    가중치 매개변수를 기울기 방향으로 아주 조금 갱신한다.   
4. 반복   
    1~3단계를 반복한다.   

오차역전파법은 매개변수가 많아도 효율적으로 계산할 수 있고 계층이 추가적으로 필요하다면 레고 블럭처럼 조립할 수 있다는 장점이 있다. 하지만 오차역전파법은 구현하기 복잡해서 종종 실수가 있을 수 있기 때문에 수치미분법을 통해 기울기 검증을 해야한다.   

In [5]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
    
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1) # output one-hot-lable
        if t.ndim != 1 : t = np.argmax(t, axis=1) # one-hot-lable

        accuray = np.sum(y==t) / float(x.shape[0])
        return accuray
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        #아래의 numerical_gradient는 common.gradient.py안에 있는 numerical_gradient
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

    def gradient(self, x, t):
        self.loss(x, t)

        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db

        return grads


In [6]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label="True")
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    grad = network.gradient(x_batch, t_batch)

    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)
    

0.11361666666666667 0.109
0.90375 0.9101
0.9279666666666667 0.9288
0.9404166666666667 0.9392
0.9460333333333333 0.9431
0.9516833333333333 0.9479
0.95705 0.9548
0.9600333333333333 0.9569
0.9636833333333333 0.9586
0.9667 0.9627
0.9689166666666666 0.9634
0.97035 0.9649
0.9732 0.9664
0.9735333333333334 0.9671
0.9762 0.9682
0.9769666666666666 0.9697
0.97815 0.9697
